#### First Implementation of scraping all events of defined Website Scrapers

In [1]:
import datetime as dt
from src import scraper_collection

end_date= dt.datetime(2023,7,27)
print(end_date)
scraper_collection.get_all_events(end_date=end_date)

2023-07-27 00:00:00
JapanischesPalais Scraper started.
Test
2023-07-24 00:00:00 ---------- 2023-07-24 10:41:41.073376
test2
2023-07-25 00:00:00 ---------- 2023-07-24 10:41:41.073376
test2
2023-07-26 00:00:00 ---------- 2023-07-24 10:41:41.073376
test2
2023-07-27 00:00:00 ---------- 2023-07-24 10:41:41.073376
test2
Datum und Zeit:  2023-07-27 13:00
                                
Titel:           Ferienprogramm: Ei Ei Ei!
Typ:             Workshops
Ort:             Japanisches Palais — Japanisches Palais 
Beschreibung:    
 Wie die Meister in Damaskus stellen wir selbst Eitemperafarben her und probieren aus, wie gut sich damit malen lässt. 
Ab 8 Jahren, max. 10 PersonenTreff: FoyerTeilnahmegebühr: kostenfrei
                    
                
        
    


ValueError: unconverted data remains: 
                                

#### qgrid example

In [ ]:
import numpy as np
import pandas as pd
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'A' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],index=list(range(9)),dtype='datetime64[ns]'),
    'B' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'C' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'D' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })
df_types['E'] = df_types['D'] == 'foo'
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=True)
qgrid_widget